<a href="https://colab.research.google.com/github/lhmpia/colab/blob/main/Catboost_ML_%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8_5%EC%B0%A8_%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install catboost

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 라이브러리
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from catboost import CatBoostClassifier

In [ ]:
# 데이터 불러오기
df = pd.read_csv("/content/data_v2 (최종).csv" , encoding='utf-8')
df

,yqc_cd,year,quarter,ds,gu_cd,gu_nm,dong_cd,dong_nm,dong_area,business_cd,...,SEL_close_avg,wp_total,rp_total,apt_cnt,apt_price,business_cat,wp_missing,rp_missing,apt_cnt_missing,apt_price_missing
0,20191,2019,1,2019-03-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,48,6476,13047,342,243371110,음식(한식),0,0,1,1
1,20191,2019,1,2019-03-01,11110,종로구,11110515,청운효자동,2568432,CS100002,...,48,6476,13047,342,243371110,음식(중식),0,0,1,1
2,20191,2019,1,2019-03-01,11110,종로구,11110515,청운효자동,2568432,CS100003,...,48,6476,13047,342,243371110,음식(일식),0,0,1,1
3,20191,2019,1,2019-03-01,11110,종로구,11110515,청운효자동,2568432,CS100004,...,48,6476,13047,342,243371110,음식(양식),0,0,1,1
4,20191,2019,1,2019-03-01,11110,종로구,11110515,청운효자동,2568432,CS100005,...,48,6476,13047,342,243371110,음식(빵집),0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404218,20244,2024,4,2024-12-01,11740,강동구,11740700,둔촌2동,1550455,CS300029,...,52,2489,24228,316,261960069,소매(일반),0,0,0,0
404219,20244,2024,4,2024-12-01,11740,강동구,11740700,둔촌2동,1550455,CS300031,...,52,2489,24228,316,261960069,생활제품,0,0,0,0
404220,20244,2024,4,2024-12-01,11740,강동구,11740700,둔촌2동,1550455,CS300033,...,52,2489,24228,316,261960069,생활서비스,0,0,0,0
404221,20244,2024,4,2024-12-01,11740,강동구,11740700,둔촌2동,1550455,CS300035,...,52,2489,24228,316,261960069,생활서비스,0,0,0,0


In [ ]:
# 데이터 정렬
df = df.sort_values(['dong_cd','business_cd','yqc_cd']).reset_index(drop=True)
df.head(10)

# 성장률(pct_change), rolling feature는 "시간 흐름" 순서가 중요
# yqc_code = 분기 코드, dong_cd = 행정동, business_nm = 업종
# 동 → 업종 → 분기 순서로 시간 흐름을 맞춰 정렬

,yqc_cd,year,quarter,ds,gu_cd,gu_nm,dong_cd,dong_nm,dong_area,business_cd,...,SEL_close_avg,wp_total,rp_total,apt_cnt,apt_price,business_cat,wp_missing,rp_missing,apt_cnt_missing,apt_price_missing
0,20191,2019,1,2019-03-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,48,6476,13047,342,243371110,음식(한식),0,0,1,1
1,20192,2019,2,2019-06-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,49,6476,13030,342,243371110,음식(한식),0,0,1,1
2,20193,2019,3,2019-09-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,48,6476,13030,342,243371110,음식(한식),0,0,1,1
3,20194,2019,4,2019-12-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,48,6476,12817,342,243371110,음식(한식),0,0,0,0
4,20201,2020,1,2020-03-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,48,6476,12817,342,243371110,음식(한식),0,0,0,0
5,20202,2020,2,2020-06-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,49,6476,12817,342,243371110,음식(한식),0,0,0,0
6,20203,2020,3,2020-09-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,49,6476,12817,342,243371110,음식(한식),0,0,0,0
7,20204,2020,4,2020-12-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,49,6252,12434,342,243371110,음식(한식),0,0,0,0
8,20211,2021,1,2021-03-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,49,6252,12434,342,243371110,음식(한식),0,0,0,0
9,20212,2021,2,2021-06-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,49,6252,12434,342,243371110,음식(한식),0,0,0,0


In [ ]:
# 성장률 계산
df["qs_growth"] = df.groupby(["dong_cd", "business_nm"])["qs_total"].pct_change()
df.head()

# 동 + 업종 내부에서 해당 분기 매출이 이전 분기 대비 얼마나 증가/감소했는지 (%)를 계산한 것
# ex) -0.1 → -10% 감소, 0.3 → 30% 증가

,yqc_cd,year,quarter,ds,gu_cd,gu_nm,dong_cd,dong_nm,dong_area,business_cd,...,wp_total,rp_total,apt_cnt,apt_price,business_cat,wp_missing,rp_missing,apt_cnt_missing,apt_price_missing,qs_growth
0,20191,2019,1,2019-03-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,6476,13047,342,243371110,음식(한식),0,0,1,1,NaN
1,20192,2019,2,2019-06-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,6476,13030,342,243371110,음식(한식),0,0,1,1,0.087986
2,20193,2019,3,2019-09-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,6476,13030,342,243371110,음식(한식),0,0,1,1,-0.031907
3,20194,2019,4,2019-12-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,6476,12817,342,243371110,음식(한식),0,0,0,0,0.289341
4,20201,2020,1,2020-03-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,6476,12817,342,243371110,음식(한식),0,0,0,0,-0.218130


In [ ]:
# 업종 대비 평균 성장률 계산
df["qs_growth_b_avg"] = df.groupby(["business_cd", "yqc_cd"])["qs_growth"].transform("mean")
df["qs_growth_b"] = df["qs_growth"] - df["qs_growth_b_avg"]
df['qs_growth_b'].head()
# 같은 업종(예: 카페) 전체의 성장률 평균은 얼마인가?
# 그 업종 평균 대비 우리 점포가 얼마나 더 잘/못했는가?
# 즉, 상대적으로 업종 시장에서 경쟁력이 있었는가? (업종 내 상대평가 지표)

,qs_growth_b
0,NaN
1,-0.022720
2,-0.048077
3,0.174848
4,-0.055394


In [ ]:
# 행정동 대비 평균 성장률 계산
df["qs_growth_d_avg"] = df.groupby(["dong_cd", "yqc_cd"])["qs_growth"].transform("mean")
df["qs_growth_d"] = df["qs_growth"] - df["qs_growth_d_avg"]
df['qs_growth_d'].head()
# 동(지역) 내 모든 업종이 평균적으로 어떻게 성장했는지 비교
# ex) 동 전체가 -10% 감소했는데, 이 점포는 -2% 감소했다.
# → 실제로는 선방했다고 봐야 함 (지역 시장 분위기 대비 상대 성과)

,qs_growth_d
0,NaN
1,-0.158638
2,-0.037832
3,-0.678168
4,-0.068879


In [ ]:
# 성장점수 계산
df["growth_score"] = (df["qs_growth_b"] * 0.5 + df["qs_growth_d"] * 0.5)
df["growth_score"]
# 업종 경쟁력과 지역 경쟁력을 반반 반영한 요약 지표.
# 시그널 정리해주는 개념.

,growth_score
0,NaN
1,-0.090679
2,-0.042954
3,-0.251660
4,-0.062137
...,...
404218,-0.096597
404219,-0.645051
404220,-0.700881
404221,-0.219106


In [ ]:
df['growth_score'].min()

-3747.3168918991446

In [ ]:
df['growth_score'].max()

367585.34753554116

In [ ]:
df["growth_score"].isna().sum()

np.int64(19018)

In [ ]:
df["growth_score"].describe()

,growth_score
count,3.852050e+05
mean,1.534693e-16
std,6.593283e+02
min,-3.747317e+03
25%,-5.184545e-01
50%,-1.822675e-01
75%,-5.913873e-03
max,3.675853e+05


In [ ]:
# 로그 성장률 계산 (성장률 기반)
df["log_growth"] = np.log1p(df["qs_growth"])
df["log_growth"]

# 성장률에는 극단값이 많음
# 갑자기 매출 0 → 10
# 5 → 100
# log1p는 극단값의 영향을 줄이는 방식.
# 이상치로부터 모델을 보호하는 안전장치.

,log_growth
0,NaN
1,0.084329
2,-0.032427
3,0.254131
4,-0.246067
...,...
404218,0.120038
404219,-0.535339
404220,-0.010441
404221,-0.120961


In [ ]:
# Rolling Feature (패턴 강화)
df["qs_roll3"] = df.groupby(["dong_cd", "business_cd"])["qs_total"].transform(lambda x: x.rolling(3).mean())
df["qs_vol"]   = df.groupby(["dong_cd", "business_cd"])["qs_total"].transform(lambda x: x.rolling(3).std())
df["qs_diff"]  = df.groupby(["dong_cd", "business_cd"])["qs_total"].transform(lambda x: x.diff())
# 3분기 이동평균, 변동성, 변화량 → 시계열 패턴을 반영하기 위한 지표들.
# 현재 catboost ml 모델은 시계열 기능이 없으므로 이런 roll 특성이 매우 중요.

In [ ]:
# rolling 값 폭발 방지: log1p 적용
df["qs_roll3"] = np.log1p(df["qs_roll3"])
df["qs_vol"] = np.log1p(df["qs_vol"].fillna(0))
df["qs_diff"] = np.log1p(df["qs_diff"].abs().fillna(0))

In [ ]:
# 업종별 Robust Scaling (업종별 정규화 작업)
def robust_norm(x):
    x_no_nan = x.dropna()
    if len(x_no_nan)==0:
        return pd.Series([np.nan]*len(x), index=x.index)
    med = x_no_nan.median()
    mad = np.median(np.abs(x_no_nan - med))
    return (x - med) / (mad + 1e-6)

df["growth_norm"] = df.groupby("business_cd")["log_growth"].transform(robust_norm)
df["growth_norm"]

,growth_norm
0,NaN
1,0.822471
2,-0.507087
3,2.756108
4,-2.939930
...,...
404218,0.397457
404219,-1.772552
404220,-0.034571
404221,-0.400513


In [ ]:
# 동 단위 상대 순위 기반 라벨 생성(추천/보통/비추천)

df["rank_pct"] = df.groupby(["dong_cd", "yqc_cd"])["growth_score"].rank(pct=True)
df["rank_pct"]

,rank_pct
0,NaN
1,0.633333
2,0.562500
3,0.700000
4,0.533333
...,...
404218,0.583333
404219,0.204082
404220,0.260000
404221,0.420000


In [ ]:
df["target"] = np.where(
    df["rank_pct"] >= 0.80, 2,     # 추천
    np.where(df["rank_pct"] <= 0.20, 0, 1)   # 비추천 , 보통
)
df["target"]

,target
0,1
1,1
2,1
3,1
4,1
...,...
404218,1
404219,1
404220,1
404221,1


In [ ]:
list(df.columns)

['yqc_cd',
 'year',
 'quarter',
 'ds',
 'gu_cd',
 'gu_nm',
 'dong_cd',
 'dong_nm',
 'dong_area',
 'business_cd',
 'business_nm',
 'qs_total',
 'qsc_total',
 'qs_weekdays',
 'qs_weekend',
 'qs_,mon',
 'qs_tue',
 'qs_wed',
 'qs_thu',
 'qs_fri',
 'qs_sat',
 'qs_sun',
 'qs_0006',
 'qs_0611',
 'qs_1114',
 'qs_1417',
 'qs_1721',
 'qs_2124',
 'qs_m',
 'qs_w',
 'qs_10',
 'qs_20',
 'qs_30',
 'qs_40',
 'qs_50',
 'qs_60',
 'qsc_weekdays',
 'qsc_weekend',
 'qsc_mon',
 'qsc_tue',
 'qsc_wed',
 'qsc_thu',
 'qsc_fri',
 'qsc_sat',
 'qsc_sun',
 'qsc_0006',
 'qsc_0611',
 'qsc_1114',
 'qsc_1417',
 'qsc_1721',
 'qsc_2124',
 'qsc_m',
 'qsc_w',
 'qsc_10',
 'qsc_20',
 'qsc_30',
 'qsc_40',
 'qsc_50',
 'qsc_60',
 'store_cnt',
 'similar_store_cnt',
 'open_rate',
 'open_cnt',
 'close_rate',
 'close_cnt',
 'frn_store_cnt',
 'government',
 'bank',
 'gen_hospital',
 'hospital',
 'phamacy',
 'kindergarden',
 'ele_school',
 'mid_school',
 'high_school',
 'university',
 'department',
 'supermarket',
 'theater',
 'accom

In [ ]:
# 피처 구성

feature_cols = [
    "qs_growth",
    "qs_growth_b",
    "qs_growth_d",
    "growth_norm",
    "qs_roll3", "qs_vol", "qs_diff",
    "store_cnt","similar_store_cnt",
    "open_cnt","close_cnt",
    "fp_total","qs_weekdays","qs_weekend",
]

feature_cols

['qs_growth',
 'qs_growth_b',
 'qs_growth_d',
 'growth_norm',
 'qs_roll3',
 'qs_vol',
 'qs_diff',
 'store_cnt',
 'similar_store_cnt',
 'open_cnt',
 'close_cnt',
 'fp_total',
 'qs_weekdays',
 'qs_weekend']

In [ ]:
# target label에서 null 값 drop
df2 = df.dropna(subset=feature_cols + ["target"])
df2.head()

,yqc_cd,year,quarter,ds,gu_cd,gu_nm,dong_cd,dong_nm,dong_area,business_cd,...,qs_growth_d_avg,qs_growth_d,growth_score,log_growth,qs_roll3,qs_vol,qs_diff,growth_norm,rank_pct,target
2,20193,2019,3,2019-09-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,0.005925,-0.037832,-0.042954,-0.032427,21.432521,18.270150,18.025893,-0.507087,0.562500,1
3,20194,2019,4,2019-12-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,0.967508,-0.678168,-0.251660,0.254131,21.540457,19.595758,20.198262,2.756108,0.700000,1
4,20201,2020,1,2020-03-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,-0.149251,-0.068879,-0.062137,-0.246067,21.532945,19.635168,20.169880,-2.939930,0.533333,1
5,20202,2020,2,2020-06-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,0.253475,-0.233436,-0.165550,0.019840,21.541490,19.586077,17.536376,0.088107,0.419355,1
6,20203,2020,3,2020-09-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,1.934341,-1.985606,-1.004522,-0.052626,21.442396,17.811444,18.495578,-0.737111,0.633333,1


In [ ]:
df_train = df2[df2["target"].notna()].copy()

print(df2.shape)
print(df_train.shape)

print("target NaN:", df2['target'].isna().sum())
print("target 분포:")
print(df_train['target'].value_counts().head())

(366434, 122)
(366434, 122)
target NaN: 0
target 분포:
target
1    219036
2     78291
0     69107
Name: count, dtype: int64


In [ ]:
X = df_train[feature_cols]
y = df_train['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,        # 클래스 불균형이면 꼭 지정
    random_state=42
)

In [ ]:
 # catboost 모델 학습
cat = CatBoostClassifier(
    iterations=1200,
    depth=8,
    learning_rate=0.02,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    random_state=42,
    verbose=200
)

cat.fit(
    X_train,
    y_train,
    eval_set=(X_test, y_test)
)

pred = cat.predict(X_test)
print(classification_report(y_test, pred))

0:	learn: 1.0707756	test: 1.0707787	best: 1.0707787 (0)	total: 337ms	remaining: 6m 44s
200:	learn: 0.2707732	test: 0.2707637	best: 0.2707637 (200)	total: 1m 7s	remaining: 5m 36s
400:	learn: 0.2493091	test: 0.2506718	best: 0.2506718 (400)	total: 2m 17s	remaining: 4m 33s
600:	learn: 0.2432664	test: 0.2459068	best: 0.2459068 (600)	total: 3m 25s	remaining: 3m 24s
800:	learn: 0.2395694	test: 0.2433962	best: 0.2433962 (800)	total: 4m 33s	remaining: 2m 16s
1000:	learn: 0.2367229	test: 0.2417792	best: 0.2417792 (1000)	total: 5m 40s	remaining: 1m 7s
1199:	learn: 0.2344188	test: 0.2407718	best: 0.2407718 (1199)	total: 6m 48s	remaining: 0us

bestTest = 0.2407717878
bestIteration = 1199

              precision    recall  f1-score   support

           0       0.86      0.86      0.86     13822
           1       0.91      0.92      0.91     43807
           2       0.89      0.86      0.88     15658

    accuracy                           0.89     73287
   macro avg       0.89      0.88      0.88

In [ ]:
# 모델 예측 평가
pred = cat.predict(X_test).flatten()
pred

array([2, 1, 2, ..., 1, 2, 2])

In [ ]:
acc = accuracy_score(y_test, pred)
acc

0.894838102255516

In [ ]:
print("===== MODEL ACCURACY =====")
print(f"Accuracy: {acc*100:.2f}%\n")

print("===== Classification Report =====")
print(classification_report(y_test, pred))

print("===== Confusion Matrix =====")
print(confusion_matrix(y_test, pred))

===== MODEL ACCURACY =====
Accuracy: 89.48%

===== Classification Report =====
              precision    recall  f1-score   support

           0       0.86      0.86      0.86     13822
           1       0.91      0.92      0.91     43807
           2       0.89      0.86      0.88     15658

    accuracy                           0.89     73287
   macro avg       0.89      0.88      0.88     73287
weighted avg       0.89      0.89      0.89     73287

===== Confusion Matrix =====
[[11854  1968     0]
 [ 1920 40312  1575]
 [    3  2241 13414]]


In [ ]:
df_pred = pd.DataFrame({
    "y_test": y_test,
    "pred": pred
})
df

,yqc_cd,year,quarter,ds,gu_cd,gu_nm,dong_cd,dong_nm,dong_area,business_cd,...,qs_growth_d_avg,qs_growth_d,growth_score,log_growth,qs_roll3,qs_vol,qs_diff,growth_norm,rank_pct,target
0,20191,2019,1,2019-03-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,1
1,20192,2019,2,2019-06-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,0.246625,-0.158638,-0.090679,0.084329,NaN,0.000000,18.955936,0.822471,0.633333,1
2,20193,2019,3,2019-09-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,0.005925,-0.037832,-0.042954,-0.032427,21.432521,18.270150,18.025893,-0.507087,0.562500,1
3,20194,2019,4,2019-12-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,0.967508,-0.678168,-0.251660,0.254131,21.540457,19.595758,20.198262,2.756108,0.700000,1
4,20201,2020,1,2020-03-01,11110,종로구,11110515,청운효자동,2568432,CS100001,...,-0.149251,-0.068879,-0.062137,-0.246067,21.532945,19.635168,20.169880,-2.939930,0.533333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404218,20234,2023,4,2023-12-01,11740,강동구,11740700,둔촌2동,1550455,CS300036,...,0.070525,0.057015,-0.096597,0.120038,15.785187,14.415373,13.796230,0.397457,0.583333,1
404219,20241,2024,1,2024-03-01,11740,강동구,11740700,둔촌2동,1550455,CS300036,...,-0.007596,-0.406933,-0.645051,-0.535339,15.782151,14.435092,15.094979,-1.772552,0.204082,1
404220,20242,2024,2,2024-06-01,11740,강동구,11740700,둔촌2동,1550455,CS300036,...,0.785004,-0.795390,-0.700881,-0.010441,15.649335,14.553061,10.873054,-0.034571,0.260000,1
404221,20243,2024,3,2024-09-01,11740,강동구,11740700,둔촌2동,1550455,CS300036,...,0.014093,-0.128025,-0.219106,-0.120961,15.394724,12.756284,13.257656,-0.400513,0.420000,1
